<a href="https://colab.research.google.com/github/ekgren/workshop/blob/main/Day1/Workshop_Primer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch

In [3]:
torch.randn(2, 3)

tensor([[ 0.5293, -0.3065,  0.7513],
        [-1.2730, -0.3234, -0.9627]])